## Report Outline

In [1]:
# import the necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
# import the dfm_basics_rate data
dfm_basics_rate = pd.read_csv('data/dfm_basics_rate.csv')
dfm_basics_rate.head()

,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,genres1,genres2,genres3
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Action,Crime,Drama
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43,Biography,Drama,NaN
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,Drama,NaN,NaN
3,tt0069204,Sabse Bada Sukh,2018,91.0,"Comedy,Drama",6.1,13,Comedy,Drama,NaN
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,Comedy,Drama,Fantasy
